## CS431/631 Big Data Infrastructure
### Fall 2019 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Vyas Anirudh Akundy
* **ID:** 20765080

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [7]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
from itertools import permutations, chain
# Now, let's tokenize Shakespeare's plays
all_tokens = []#list of all tokens
all_tokenpairs_list = []#list of all token-pairs
with open('Shakespeare.txt') as f:

    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        all_tokenpairs_list.append(list(permutations(list(set(t)),2)))#append all possible token-pairs
        for token in t:
            all_tokens.append(token)#append all tokens
    all_tokenpairs_list = list(chain.from_iterable(all_tokenpairs_list))
    print("Number of distinct tokens: " + str(len(set(all_tokens))))#distinct tokens
    print("Number of distinct token pairs: " + str(len(set(all_tokenpairs_list))))#distinct token-pairs
# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

Number of distinct tokens: 25975
Number of distinct token pairs: 1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [9]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log
from itertools import permutations, chain
from collections import Counter

#If this cell takes more than 15 seconds, please stop it and run again. It will take 15 seconds to run
total_lines = 0
one_count = {}#stores frequency of each token
two_count = {}#stores frequency of all token-pairs
prob_OneCount = {}#stores probabilities of single token
prob_TwoCount = {}#stores probabilities of all token-pairs
PMI = {}#stores the PMI values of all token-pairs
with open('Shakespeare.txt') as f:
    lines_SingleList = []
    lines_PairsList= []
    
    for line in f:
        # tokenize, one line at a time
        total_lines = total_lines + 1
        t = simple_tokenize(line)
        single = list(set(t))#get distinct tokens
        lines_SingleList.append(single)
        pairs = list(permutations(single,2))#get distinct token pairs
        lines_PairsList.append(pairs)
    #count the frequency for each and calculate probabilities and PMI values
    one_count = dict(Counter(chain.from_iterable(lines_SingleList)))
    two_count = dict(Counter(chain.from_iterable(lines_PairsList)))
    prob_OneCount = {k:v/total_lines for (k,v) in one_count.items()}
    prob_TwoCount = {k:v/total_lines for (k,v) in two_count.items()}
    PMI = {k:log(v/(prob_OneCount[k[0]]*prob_OneCount[k[1]]),10) for (k,v) in prob_TwoCount.items()}
    
###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        if q_tokens[0] not in all_tokens:#check if token exists in document
            print("\'"+str(q_tokens[0]) +"\'"+" is not in the document, please re-enter")
            continue
        threshold = 0
        while threshold <= 0:#enter correct threshold number
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
        
        #temporary dictionaries to store count and PMI values of top 5 co-occuring tokens
        temp_count = {}
        temp_pmi = {}
        for k,v in two_count.items():
            if q_tokens[0]==k[0]:#check if token is the first value in tuple
                if v>=threshold:
                    temp_count[k] = v
                    temp_pmi[k] = PMI[k]
        if len(temp_pmi)>=5:
            temp_pmi = dict(sorted(temp_pmi.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)[0:5])
        elif len(temp_pmi)==0:#restart if threshold is too high
            print("Threshold is too high for " + "\'"+str(q_tokens[0]) +"\'" + ", please re-enter")
            continue
        else:#if there are fewer than 5 then print fewer than 5
            temp_pmi = dict(sorted(temp_pmi.items(), key = lambda kv:(kv[1], kv[0]), reverse = True))
        print("  n({0}) = {1}".format(q_tokens[0],one_count[q_tokens[0]]))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
        for k,v in temp_pmi.items():
            print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0],k[1],temp_count[k],v))    
            

    elif len(q_tokens) == 2:
        if q_tokens[0] in all_tokens:#check if tokens exist and prompt if doesnt exist
            if q_tokens[1] in all_tokens:
                if (q_tokens[0],q_tokens[1]) in all_tokenpairs_list:#print the count and PMI values
                    print("  n({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],two_count[(q_tokens[0],q_tokens[1])]))
                    print("  PMI({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],PMI[(q_tokens[0],q_tokens[1])]))
                else:#restart if pair doesnt exist
                    print("The pair ""\'"+str((q_tokens[0],q_tokens[1])) +"\'"+" do no co-occur but exist in the document, please re-enter")
                    continue
            else:#one of the tokens doesnt exist
                print("\'"+str(q_tokens[1]) +"\'" + " is not in the document, please re-enter")
                continue
        else:#one of the tokens doesnt exist
            print("\'"+str(q_tokens[0]) +"\'" + " is not in the document, please re-enter")
            continue
                
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


Input 1 or 2 space-separated tokens (return to quit): the
Input a positive integer frequency threshold: 500
  n(the) = 24300
  high PMI tokens with respect to the (threshold: 500):
    n(the,duke) = 501,  PMI(the,duke) = 0.3818573318248373
    n(the,of) = 7266,  PMI(the,of) = 0.34294075191889295
    n(the,king) = 1117,  PMI(the,king) = 0.30172774181582845
    n(the,from) = 754,  PMI(the,from) = 0.16561487229059674
    n(the,upon) = 500,  PMI(the,upon) = 0.1640585553738594
Input 1 or 2 space-separated tokens (return to quit): the spiderman
'spiderman' is not in the document, please re-enter
Input 1 or 2 space-separated tokens (return to quit): bye bye
The pair '('bye', 'bye')' do no co-occur but exist in the document, please re-enter
Input 1 or 2 space-separated tokens (return to quit): at time
  n(at,time) = 65
  PMI(at,time) = 0.4869482942671367
Input 1 or 2 space-separated tokens (return to quit): william shakspear
'shakspear' is not in the document, please re-enter
Input 1 or 2 spac

---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

 - As the input file grows, the program will take much more time to run(which increases linearly with the number of lines in the file), but it may not fail to run for smaller inputs. If the input is really huge(in petabytes) then the program may fail as the system may not have enough memory to run the program.
 - The data structures used here hold all the words in the file. If the text contains billions and billions of words, it may exceed the capacity and we may need more machines to share the load


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.